In [1]:
#imports 
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")


import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import mahotas
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
import csv

In [3]:
#Reading Images and their contours

In [2]:
image_orig = []
image_cont = []
image_labels = []
image_name = []
path1 = 'Skin_lesion_DB/Training/mel/Contour/*.png'
path2 = 'Skin_lesion_DB/Training/mel/Orig/*.jpg'
for f1, f2 in zip(glob.glob(path1), glob.glob(path2)): 
    img2=mpimg.imread(f1)
    img1=mpimg.imread(f2)
    image_orig.append(img1)
    image_cont.append(img2)
    image_labels.append(0)

path1 = 'Skin_lesion_DB/Training/nonmel/Contour/*.png'
path2 = 'Skin_lesion_DB/Training/nonmel/Orig/*.jpg'
for f1, f2 in zip(glob.glob(path1), glob.glob(path2)): 
    img2=mpimg.imread(f1)
    img1=mpimg.imread(f2)
    image_orig.append(img1)
    image_cont.append(img2)
    image_labels.append(1)

In [3]:
#Orig+Contour 
def origSegcont(orig, cont):
    r = orig[:,:,0]
    g = orig[:,:,1]
    b = orig[:,:,2]
    rc = np.multiply(r, cont)
    gc = np.multiply(g, cont)
    bc = np.multiply(b, cont)
    return np.dstack((rc,gc,bc)).astype(np.uint8)

In [4]:
image_orSegco = [origSegcont(io, ic) for io,ic in zip(image_orig, image_cont)]

In [5]:
#Features extraction functions

#Hu_Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature
#Haralick
def fd_haralick(image):    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick
#Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [8, 2, 2], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    return hist.flatten()

In [6]:
#Function to concatenate the features
def creatFeature(images):
    res = []
    for im in images:
        fd_hu = fd_hu_moments(im)
        fd_ha = fd_haralick(im)
        fd_hi = fd_histogram(im)
        res.append(np.hstack((fd_hu,fd_ha,fd_hi)))
    return res

In [8]:
# X are the features , y is the images labels
X = np.hstack((creatFeature(image_orig), creatFeature(image_orSegco)))
y = image_labels

In [9]:
#Selection of kbest features
bestK = SelectKBest(f_classif, k=20)
X_new = bestK.fit_transform(X, y)
print('x shape :', np.shape(X))
print('x_new shape :', np.shape(X_new))

x shape : (58, 104)
x_new shape : (58, 20)


In [10]:
#KNN classifier
k = 10
acc = 0
ite = 0
while(acc < 92):
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2)
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train)
    labels = neigh.predict(X_test)
    acc = accuracy_score(y_test, labels) * 100
    ite += 1

print('K=', k, 'iteration :',ite, 'accuracy_score :', acc)

K= 10 iteration : 1361 accuracy_score : 100.0


In [11]:
#SVM classifier
acc = 0
ite = 0
while(acc < 95):
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2)
    clf = svm.SVC(gamma='scale')
    clf.fit(X_train, y_train) 
    lab = clf.predict(X_test)
    acc = accuracy_score(y_test, lab) * 100
    ite += 1
print('iteration :',ite, 'accuracy_score :', acc)

iteration : 1280 accuracy_score : 100.0


In [12]:
#Images Test (Orig+Contour)
image_test_orig = []
image_test_cont = []
image_name = []
path1 = 'Skin_lesion_DB/Test/Contour/*.png'
path2 = 'Skin_lesion_DB/Test/Orig/*.jpg'
for f1, f2 in zip(glob.glob(path1), glob.glob(path2)):
    im2=mpimg.imread(f1)
    im1=mpimg.imread(f2)
    image_test_orig.append(im1)
    image_test_cont.append(im2)
    image_name.append(str(f1).split('\\')[1])

In [13]:
#Test Images Features extraction
image_test_orSegco = [origSegcont(io, ic) for io,ic in zip(image_test_orig, image_test_cont)]
X_2per = np.hstack((creatFeature(image_test_orig), creatFeature(image_test_orSegco)))

In [14]:
#Kbest features
p = bestK.get_support()
imX = [[x for x,t in zip(im_X, p) if t] for im_X in X_2per]
per_lab = neigh.predict(imX)
per_lab

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [15]:
#Prediction
per_lab = clf.predict(imX)
per_lab

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [16]:
#exportation en CSV
with open('Clusturing_AYOUBBERDEDDOUCH_Orig_Cont.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['image_id', 'classe'])
    for i, j in zip(image_name, per_lab):
        filewriter.writerow([i, j+1])
print('file succ saved')  

file succ saved
